In [ ]:
#如果不能通过检验（>0.1）就需要尝试分箱或者log化
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import gc
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS)

columns = X.drop(cats,axis=1).columns
splits = folds.split(X, y)

PSI = pd.DataFrame()
PSI['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid =X.iloc[train_index],X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]    
    PSI[f'fold_{fold_n + 1}'] = toad.metrics.PSI(X_train.drop(cats,axis=1),X_valid.drop(cats,axis=1)).values       
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
PSI

In [ ]:
#计算psi的函数，用到toad包
def PSI_cal(NFOLDS,X,y,cats):
    NFOLDS = NFOLDS
    folds = StratifiedKFold(n_splits=NFOLDS)

    columns = X.drop(cats,axis=1).columns
    splits = folds.split(X, y)

    PSI = pd.DataFrame()
    PSI['feature'] = columns

    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid =X.iloc[train_index],X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]



        #PSI[f'fold_{fold_n + 1}'] = toad.metrics.PSI(X_train.drop(cats,axis=1),X_valid.drop(cats,axis=1)).values

        PSI[f'fold_{fold_n + 1}'] = toad.metrics.PSI(X_train.TransactionAmt,X_valid.TransactionAmt)
        del X_train, X_valid, y_train, y_valid
        gc.collect()
    return PSI.drop('feature',axis=1).iloc[0].mean()

In [ ]:
#先计算原始iv和psi值
print(toad.quality(X[['TransactionAmt','y']],'y').iv.values[0])
print(PSI_cal(5,X,y,cats))

In [ ]:
#log化
TransactionAmt=X.TransactionAmt
X.TransactionAmt=np.log1p(X.TransactionAmt)

In [ ]:
#决策树分箱
iv=[]
PSIs=[]
for bins in [5,10,15,20,25,30,35,40,45,50]:
    bins=toad.DTMerge(TransactionAmt,y,n_bins=bins).tolist()
    bins.insert(0,-np.inf)
    bins.append(np.inf)
    X.TransactionAmt=np.digitize(TransactionAmt,bins)
    iv.append(toad.quality(X[['TransactionAmt','y']],'y').iv.values[0])
    PSIs.append(PSI_cal(5,X,y,cats))

In [ ]:
#卡方分箱，一般来说效果最好
iv=[]
PSIs=[]
for bins in [5,10,15,20,25,30,35,40,45,50]:
    bins=toad.ChiMerge(TransactionAmt,y,n_bins=bins).tolist() # DTmerge，ChiMerge，KMeansMerge，QuantileMerge
    bins.insert(0,-np.inf)
    bins.append(np.inf)
    X.TransactionAmt=np.digitize(TransactionAmt,bins)
    iv.append(toad.quality(X[['TransactionAmt','y']],'y').iv.values[0])
    PSIs.append(PSI_cal(5,X,y,cats))

In [ ]:
#kmeans分箱
iv=[]
PSIs=[]
for bins in [5,10,15,20,25,30,35,40,45,50]:
    bins=toad.KMeansMerge(TransactionAmt,y,n_bins=bins).tolist() # DTmerge，ChiMerge，KMeansMerge，QuantileMerge
    bins.insert(0,-np.inf)
    bins.append(np.inf)
    X.TransactionAmt=np.digitize(TransactionAmt,bins)
    iv.append(toad.quality(X[['TransactionAmt','y']],'y').iv.values[0])
    PSIs.append(PSI_cal(5,X,y,cats))

In [ ]:
#查看分箱之后的效果
bins=toad.ChiMerge(TransactionAmt,y,n_bins=10).tolist() # DTmerge，ChiMerge，KMeansMerge，QuantileMerge
bins.insert(0,-np.inf)
bins.append(np.inf)
X.TransactionAmt=np.digitize(TransactionAmt,bins)
sns.kdeplot(X.TransactionAmt)
print(toad.quality(X[['TransactionAmt','y']],'y').iv.values[0])
print(PSI_cal(5,X,y,cats))

In [ ]:
#可以把原始变量也放进去抵消
X['original_TransactionAmt']=TransactionAmt